# Semantic Segmentation: How to run inference on the endpoint you have created?

### Download example image

In [1]:
import boto3
import numpy as np
from IPython.core.display import HTML

region = boto3.Session().region_name
s3_bucket = f"jumpstart-cache-prod-{region}"
key_prefix = "inference-notebook-assets"
s3 = boto3.client("s3")

def download_from_s3(key_filenames):
    for key_filename in key_filenames:
        s3.download_file(s3_bucket, f"{key_prefix}/{key_filename}", key_filename)
        
img_jpg = "img_pedestrian.png"
download_from_s3(key_filenames=[img_jpg])

HTML('<img src="img_pedestrian.png" alt="Pedestrian Image" style="height: 600px;"/>'
     '<figcaption>img_pedestrian.png</figcaption>The image has been downloaded from '
     'https://www.cis.upenn.edu/~jshi/ped_html/')

### Query endpoint that you have created

In [2]:
import json
def query_endpoint(input_img):
    endpoint_name = 'jumpstart-dft-mx-semseg-fcn-resnet101-coco'
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/x-image', Body=input_img, Accept='application/json;verbose')
    return response

def parse_response(query_response):
    response_dict = json.loads(query_response['Body'].read())
    return response_dict['predictions'],response_dict['labels'], response_dict['image_labels']

with open(img_jpg, 'rb') as file: input_img = file.read()

try:
    query_response = query_endpoint(input_img)
except Exception as e:
    if e.response['Error']['Code'] == 'ModelError':
        raise Exception(
             "Backend scripts have been updated in Feb '22 to standardize response "
             "format of endpoint response."
             "Previous endpoints may not support verbose response type used in this notebook."
             f"To use this notebook, please launch the endpoint again. Error: {e}."
        )
    else:
        raise
try:
    predictions, labels, image_labels =  parse_response(query_response)
except (TypeError, KeyError) as e:
    raise Exception(
          "Backend scripts have been updated in Feb '22 to standardize response "
          "format of endpoint response."
           "Response from previous endpoints not consistent with this notebook."
           f"To use this notebook, please launch the endpoint again. Error: {e}."
   )

### Objects present in the image.

In [3]:
print('Objects present in the picture:',image_labels)

Objects present in the picture: ['background', 'person']


### Get color palette for visualization.

We borrow the VOC pallete implementation from [GluonCV](https://cv.gluon.ai/_modules/gluoncv/utils/viz/segmentation.html#get_color_pallete)  

In [4]:
def getvocpallete(num_cls):
    n = num_cls
    pallete = [0]*(n*3)
    for j in range(0, n):
        lab = j
        pallete[j*3+0] = 0
        pallete[j*3+1] = 0
        pallete[j*3+2] = 0
        i = 0
        while (lab > 0):
            pallete[j*3+0] |= (((lab >> 0) & 1) << (7-i))
            pallete[j*3+1] |= (((lab >> 1) & 1) << (7-i))
            pallete[j*3+2] |= (((lab >> 2) & 1) << (7-i))
            i = i + 1
            lab >>= 3
    return pallete

pallete = getvocpallete(256)

### Add color pallete to prediction for visualization and show the predicted masks.

In [5]:
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

npimg = np.array(predictions)
npimg[npimg == -1] = 255
mask = Image.fromarray(npimg.astype('uint8'))
mask.putpalette(pallete)
mask.save('Mask_putput.png')
mmask = mpimg.imread('Mask_putput.png')
plt.imshow(mmask)

In [9]:
# Convert the pedestrian Image to JPEG so we get the pixel details
im1 = Image.open(r'img_pedestrian.png')
rgb_im1 = im1.convert('RGB')
rgb_im1.save('img_pedestrian-1.jpg')

# Convert the mask output to JPEG so we get the pixel details
im2 = Image.open(r'Mask_putput.png')
rgb_im2 = im2.convert('RGB')
rgb_im2.save('Mask_putput-1.jpg')

im1 = Image.open('img_pedestrian-1.jpg')
pixelMap1 = im1.load()
im2 = Image.open('Mask_putput-1.jpg')
pixelMap2 = im2.load()
width1, height1 = im1.size
width2, height2 = im2.size


# Set the height and width to the largest image

if width2 > width1:
    width = width2
else:
    width = width1

if height2 > height1:
    height = height2
else:
    height = height1
    
#print(width)
#print(height)
#for i in range(img.size[0]):    # for every col:
#    for j in range(img.size[1]):    # For every row
#        pixelMap[i,j] = pixelMap2[i,j] # set the colour accordingly
for i in range(width):    # for every col:
    for j in range(height):    # For every row
        #print(pixelMap[i,j])
        #print(pixelMap2[i,j])
        R1, G1, B1 = pixelMap2[i,j]
        if R1!=0 and G1!=0 and B1!=0:
            pixelMap2[i,j]=pixelMap1[i,j]
        else:
            pixelMap2[i,j]=(0,0,0)
            
              
im1.show()
im2 = im2.save("Final_Pedestrians.jpg")